In [1]:
import pandas as pd
import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot

init_notebook_mode(connected=True)

In [55]:
#import dataframe
df=pd.read_csv("Dataset/dataset_superstore_simple.csv",parse_dates=["order_date"])
df.head()

order_id order_date customer_id    segment         category  \
0  CA-2016-152156 2016-11-08    CG-12520   Consumer        Furniture   
1  CA-2016-152156 2016-11-08    CG-12520   Consumer        Furniture   
2  CA-2016-138688 2016-06-12    DV-13045  Corporate  Office Supplies   
3  US-2015-108966 2015-10-11    SO-20335   Consumer        Furniture   
4  US-2015-108966 2015-10-11    SO-20335   Consumer  Office Supplies   

  sub_category     sales  quantity    profit  
0    Bookcases  261.9600         2   41.9136  
1       Chairs  731.9400         3  219.5820  
2       Labels   14.6200         2    6.8714  
3       Tables  957.5775         5 -383.0310  
4      Storage   22.3680         2    2.5164

In [56]:
#customer dengan sales terbesar
df_sales_terbesar=df[["customer_id","sales"]]
df_sales_terbesar.sort_values("sales",ascending=False)

#df.groupby(['customer_id'])['sales'].max()


customer_id     sales
9033    AB-10105  9892.740
4096    SC-20095  9449.950
4274    BS-11365  9099.930
8482    SE-20110  8749.950
6422    CC-12370  8399.976
...          ...       ...
2106    HG-14965     0.876
4708    JO-15280     0.852
8652    KB-16600     0.836
9286    RS-19765     0.556
4099    ZC-21910     0.444

[9988 rows x 2 columns]

In [57]:
#Total profit subcategory dalam Office Supplies
df_groupby=df.groupby(["category","sub_category"]).agg({
    "profit":"sum",
})
df_groupby.loc["Office Supplies"]

profit
sub_category            
Appliances    18138.0054
Art            6527.7870
Binders       30221.7633
Envelopes      6964.1767
Fasteners       949.5182
Labels         5546.2540
Paper         34053.5693
Storage       21278.8264
Supplies      -1189.0995

In [43]:
import numpy as np

In [58]:
#Banyak order dengan profit negatif (yang true=1869)
df_rugi=df["profit"]<0
df_rugi.value_counts()

False    8119
True     1869
Name: profit, dtype: int64

In [59]:
#total sales paling banyak antara 3 customer
df_groupby=df.groupby(["customer_id"]).agg({
    "sales":"sum",
})
df_groupby.loc[["JE-16165","KH-16510","AD-10180"]]

sales
customer_id          
JE-16165     2697.248
KH-16510     5952.859
AD-10180     6106.880

In [65]:
#tahun dengan profit tertinggi
df["year"]=df.order_date.dt.strftime("%Y")
df_yearly_sales=df.groupby(["year"]).agg({
    "sales":"sum",
    "profit":"sum",
})
df_yearly_sales=df_yearly_sales.rename(columns={
    "sales":"total sales",
    "profit":"total profit",
})
df_yearly_sales


total sales  total profit
year                           
2014  461609.0181    51355.0525
2015  470532.5090    61618.6037
2016  587205.6630    79995.1763
2017  697515.3572    77759.3144

In [113]:
df_sales_1415=df[["year","sales","profit"]]
df_sales_1415=df_sales_1415[df_sales_1415["year"].isin(["2014","2015"])]
df_sales_1415

year     sales    profit
3     2015  957.5775 -383.0310
4     2015   22.3680    2.5164
5     2014   48.8600   14.1694
6     2014    7.2800    1.9656
7     2014  907.1520   90.7152
...    ...       ...       ...
9965  2015  140.7500   42.2250
9974  2015   85.9800   22.3548
9978  2015   31.5000   15.1200
9979  2015   55.6000   16.1240
9983  2014   25.2480    4.1028

[4094 rows x 3 columns]

In [124]:
#Scatterplot
dat1=go.Scatter(
    x=df_sales_1415[df_sales_1415.year == "2014"].sales,
    y=df_sales_1415[df_sales_1415.year == "2014"].profit,
    mode="markers",
    name="Tahun 2014"
)

dat2=go.Scatter(
    x=df_sales_1415[df_sales_1415.year == "2015"].sales,
    y=df_sales_1415[df_sales_1415.year == "2015"].profit,
    mode="markers",
    name="Tahun 2015"
)


layout={
    "title":{
        "text":"Sales VS Profit 2014-2015",
        "x":0.5,
    },
    "xaxis":{
        "title":"Sales",
        "range":[0,7500]
    },
    "yaxis":{
        "title":"Profit",
        "range":[-2500,2500]
    }
}

data=[dat1,dat2]
fig=go.Figure(data=data,layout=layout)
iplot(fig)

In [125]:
#Barchart
#10 customer dengan sales tertinggi
df_2015=df[["year","customer_id","sales","profit"]]
df_2015=df_2015[df_2015["year"].isin(["2015"])]
df_2015

year customer_id      sales    profit
3     2015    SO-20335   957.5775 -383.0310
4     2015    SO-20335    22.3680    2.5164
14    2015    HP-14815    68.8100 -123.8580
15    2015    HP-14815     2.5440   -3.8160
24    2015    EB-13870  1044.6300  240.2649
...    ...         ...        ...       ...
9964  2015    MP-17470   119.5600   54.9976
9965  2015    MP-17470   140.7500   42.2250
9974  2015    SW-20455    85.9800   22.3548
9978  2015    DV-13465    31.5000   15.1200
9979  2015    DV-13465    55.6000   16.1240

[2102 rows x 4 columns]

In [134]:
df_customer_sales_tertinggi=df_2015.groupby(["customer_id"]).agg({
    "sales":"sum",
    "profit":"sum",
})
df_customer_sales_tertinggi=df_customer_sales_tertinggi.sort_values("sales",ascending=False)
df_customer_sales_tertinggi=df_customer_sales_tertinggi.reset_index()
df_customer_sales_tertinggi=df_customer_sales_tertinggi.head(10)
df_customer_sales_tertinggi

customer_id     sales     profit
0    PF-19120  9022.324  -631.9789
1    KD-16495  7039.044  2810.0215
2    CM-12385  6682.260  3197.4580
3    FH-14365  6056.090  2139.7363
4    NW-18400  5511.316  1654.2153
5    SB-20185  5414.246   939.1590
6    JH-15985  5352.786 -1429.3776
7    HM-14860  5327.802  2450.9987
8    CB-12025  4662.553   223.4447
9    QJ-19255  4409.850  1014.4635

In [135]:
data=go.Bar(
    x=df_customer_sales_tertinggi.customer_id,
    y=df_customer_sales_tertinggi.profit,
)
fig = go.Figure(data=data)
iplot(fig)